In [1]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import accuracy
from surprise.prediction_algorithms.knns import *
%matplotlib inline

In [2]:
ratings = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/ratings.csv', sep = ',', engine = 'python' )

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
ratings.count()

userId       26024289
movieId      26024289
rating       26024289
timestamp    26024289
dtype: int64

In [5]:
movies = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/movies.csv', sep = ',', engine = 'python' )

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.count()

movieId    45843
title      45843
genres     45843
dtype: int64

In [8]:
tags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/tags.csv', sep = ',', engine = 'python' )

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,1,318,narrated,1425942391
1,20,4306,Dreamworks,1459855607
2,20,89302,England,1400778834
3,20,89302,espionage,1400778836
4,20,89302,jazz,1400778841


In [10]:
tags.count()

userId       753170
movieId      753170
tag          753154
timestamp    753170
dtype: int64

In [11]:
links = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/links.csv', sep = ',', engine = 'python' )

In [12]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
links.count()

movieId    45843
imdbId     45843
tmdbId     45624
dtype: int64

In [14]:
genomeScores = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-scores.csv', sep = ',', engine = 'python' )

In [15]:
genomeScores.head()

,movieId,tagId,relevance
0,1,1,0.02475
1,1,2,0.02475
2,1,3,0.04900
3,1,4,0.07750
4,1,5,0.12450


In [16]:
genomeScores.count()

movieId      12400104
tagId        12400104
relevance    12400104
dtype: int64

In [17]:
genomeTags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-tags.csv', sep = ',', engine = 'python' )

In [18]:
genomeTags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [19]:
genomeTags.count()

tagId    1128
tag      1128
dtype: int64

In [20]:
tags['DateTime'] = pd.to_datetime(tags['timestamp'], unit='s')

In [21]:
tags.head()

,userId,movieId,tag,timestamp,DateTime
0,1,318,narrated,1425942391,2015-03-09 23:06:31
1,20,4306,Dreamworks,1459855607,2016-04-05 11:26:47
2,20,89302,England,1400778834,2014-05-22 17:13:54
3,20,89302,espionage,1400778836,2014-05-22 17:13:56
4,20,89302,jazz,1400778841,2014-05-22 17:14:01


In [22]:
ratings['DateTime'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [23]:
ratings.head()

,userId,movieId,rating,timestamp,DateTime
0,1,110,1.0,1425941529,2015-03-09 22:52:09
1,1,147,4.5,1425942435,2015-03-09 23:07:15
2,1,858,5.0,1425941523,2015-03-09 22:52:03
3,1,1221,5.0,1425941546,2015-03-09 22:52:26
4,1,1246,5.0,1425941556,2015-03-09 22:52:36


In [24]:
date_rating = pd.DataFrame(ratings['DateTime']).drop_duplicates()

In [25]:
date_rating.describe()

,DateTime
count,20549435
unique,20549435
top,2013-04-24 11:28:21
freq,1
first,1995-01-09 11:46:44
last,2017-08-04 06:57:50


In [26]:
users_rating = pd.DataFrame(ratings['userId']).drop_duplicates()

In [27]:
users_rating.describe()

,userId
count,270896.000000
mean,135448.500000
std,78201.083599
min,1.000000
25%,67724.750000
50%,135448.500000
75%,203172.250000
max,270896.000000


In [28]:
genomes = pd.merge(genomeScores,genomeTags, on=['tagId'])

In [29]:
genomes.head()

,movieId,tagId,relevance,tag
0,1,1,0.02475,007
1,2,1,0.03750,007
2,3,1,0.03800,007
3,4,1,0.03125,007
4,5,1,0.04100,007


In [30]:
genomes.count()

movieId      12400104
tagId        12400104
relevance    12400104
tag          12400104
dtype: int64

In [31]:
pelicula = pd.DataFrame(tags[tags['movieId']==1]['tag']).drop_duplicates()
pelicula.sort_values('tag')

,tag
657468,2009 reissue in Stereoscopic 3-D
614820,3D
657469,55 movies every kid should see--Entertainment ...
736440,American Animation
355068,Animation
657470,BD-Video
706177,Best of Rotten Tomatoes: All Time
686606,Buzz Lightyear
686607,CG animation
510255,CGI


In [32]:
pelicula = movies[movies['movieId']==781]
pelicula

,movieId,title,genres
768,781,Stealing Beauty (1996),Drama
